# Regelungstechnik

In [ ]:
#| code-fold: true


#import sys
# adding Folder_2 to the system path
#sys.path.insert(0, '../../../Common/QuartoBookHelpers')
#import QuartoBookHelpers

from IPython.display import Markdown, Latex

from sympy import *
import sympy.physics.units as u
import numpy as np
from matplotlib import pyplot as plt
from quantiphy import Quantity
import re
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal

class MySymbol(Symbol):
    def __new__(cls, name, description='',unit='',value='',**kwargs):
        obj = Symbol.__new__(cls, name,**kwargs)
        obj.description = description
        obj.unit = unit
        obj.value = value
        
        return obj

class MyFunction(Function):
    def __new__(cls, name, description='',unit='',value=''):
        obj = Function.__new__(cls, name)
        obj.description = description
        obj.unit = unit
        obj.value = value
        
        return obj
        
class QBookHelpers():
    def print_description(dic):
        for vari in dic:
            display(Markdown(f'${latex(vari)}$' + ' ... ' + vari.description))
            
    def print_equation(eq,**kwargs):
        #print(kwargs)
        if kwargs != {}:
            label = kwargs['label']
            #print(label)
            display(Markdown(f'$$\n{latex(eq)}\n$${{#' + label + f'}}'))
        else:      
            display(Markdown(f'$$\n{latex(eq)}\n$${{#eq-Dummy}}'))

    def print_values2(dic,**kwargs):
        for vari in dic:
            QuantValue = Quantity(vari.value)
            #print(QuantValue)
            quant_string = QuantValue.render(prec=2)
            #print(quant_string)
            val = re.split('(\d+)', quant_string)[1]
            if re.split('(\d+)', quant_string)[2] == '.':
                val = val + re.split('(\d+)', quant_string)[2] + re.split('(\d+)', quant_string)[3]
            #print(val)
            try:
                prefix = re.split('(\d+)', quant_string)[-1]
            except:
                prefix = ''
            #print(prefix)
            uni = vari.unit
            disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' \ ' + latex(prefix) + latex(uni) + f'$$' 
            display(Markdown(disp_string))
            
    def print_values(dic,**kwargs):
        for vari in dic:
            try:
                QuantValue = Quantity(vari.value,str(vari.unit.abbrev))
            except:
                QuantValue = Quantity(vari.value)
            quant_string = QuantValue.render(prec=2)
            val = quant_string.split(' ')[0]
            if quant_string[-1].isdigit():
                uni = ''
            else:
                try:
                    uni = quant_string.split(' ')[1]
                except:
                    uni = quant_string[-1]
            #print(re.split('(\d+)',QuantValue.render(prec=2)))
            #uni = re.split('(\d+)',QuantValue.render(prec=2))[-1]
            #uni = ''
            #val = re.split('(\d+)',QuantValue.render(prec=2))[1]
            #val = QuantValue.render(prec=2)

            if uni.find('ohm') >=0:
                uni = '\ \mathrm{' + uni.replace('ohm','\Omega') + '}'
                disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' ' + str(uni) + f'$$' 
            else:
                disp_string = f'$${latex(vari)}' + ' = ' + latex(val) + ' \ ' + latex(uni) + f'$$' 
            
            if kwargs != {}:
                label = kwargs['label']
                disp_string = disp_string + f'{{#' + label + f'}}'

            
            display(Markdown(disp_string))

            #display(Markdown(f'${latex(vari)}$' + ' = ' + str(vari.value) + f'$\ {latex(vari.unit)}$' ))            
            #display(Markdown(f'${latex(vari)}$' + ' = ' + str("%.2f" % round(vari.value, 2)) + f'$\ {latex(vari.unit)}$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + latex(QuantValue.render()) + f'$$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + latex(quant) + f'$$' ))
            #display(Markdown(f'$${latex(vari)}' + ' = ' + str(vari.value) + f'\ {latex(vari.unit)}$$' ))

            
    def calculate_num_value(eqn):
        replacement = {}
        for sym in eqn.rhs.free_symbols:
            try:
                case = {sym:sym.value}
                replacement.update(case)
            except:
                pass
        #print(replacement)
        #return Eq(eqn.lhs,eqn.rhs.evalf(subs=replacement))
        eqn.lhs.value = eqn.rhs.evalf(subs=replacement)
        #display(eqn.lhs.value)

    def replace_num_value(eqn):
        replacement = {}
        for sym in eqn.rhs.free_symbols:
            try:
                case = {sym:sym.value}
                replacement.update(case)
            except:
                pass
        #print(replacement)
        try:
            eqn = Eq(eqn.lhs,eqn.rhs.evalf(subs=replacement))
        except:
            eqn = Eq(eqn.lhs,eqn.rhs.subs(subs=replacement))
        return eqn

    def logic_parser(eqn):
        return str(eqn).replace('&','and').replace('|','or').replace('~','not')

    def trans_plot_matplotlib(filepath, thing_to_plot):
        #https://matplotlib.org/stable/gallery/ticks/tick-formatters.html
        from matplotlib.ticker import FuncFormatter
        time_formatter = FuncFormatter(lambda v, p: str(Quantity(v, 's')))
        volt_formatter = FuncFormatter(lambda v, p: str(Quantity(v, 'V')))

        df = pd.read_csv(filepath,sep=';',decimal=',')
        for key in thing_to_plot:
            suplotnumber_max = 0
            if max(thing_to_plot[key]) > 0:
                suplotnumber_max = max(thing_to_plot[key])
        
        fig, ax = plt.subplots(suplotnumber_max,1,figsize=(5, 3*suplotnumber_max))
        for i, key in enumerate(thing_to_plot):
            #for i in thing_to_plot[key]:
            ax[i].plot(df.time, df[key],label=key)
            ax[i].set_xlabel('Time in s')
            ax[i].set_ylabel(key)
            ax[i].grid()
            ax[i].legend()
            ax[i].xaxis.set_major_formatter(time_formatter)
            ax[i].yaxis.set_major_formatter(volt_formatter)
            ax[i].yaxis_tickformat = ".3s"
            ax[i].xaxis_tickformat = ".3s"

        plt.tight_layout()
        plt.savefig(filepath.replace('.csv','.svg'))
        plt.show()
        

    def trans_plot(filepath, thing_to_plot):
        df = pd.read_csv(filepath,sep=';',decimal=',')

        for key in thing_to_plot:
            suplotnumber_max = 0
            if max(thing_to_plot[key]) > 0:
                suplotnumber_max = max(thing_to_plot[key])

        fig = make_subplots(
            rows=suplotnumber_max, cols=1,
        )

        for key in thing_to_plot:
            for i in thing_to_plot[key]:
                fig.add_trace(go.Scatter(x=df.time.values, y=df[key].values,name=key), row=thing_to_plot[key][0], col=1)
                fig.update_xaxes(title_text=None, tickformat = ".3s", ticksuffix="s",  row=thing_to_plot[key][0], col=1)
                if key[0] == 'v':
                    ticksuff = 'V'
                elif key[0] == 'i':
                    ticksuff = 'A'
                else:
                    ticksuff = ''
                fig.update_yaxes(title=key, ticksuffix=ticksuff,tickformat = ".3s", row=thing_to_plot[key][0], col=1)

        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.15,
                xanchor="left",
                x=0.01
            ),
            height=1000/8*suplotnumber_max,
            width=800,
        )

        fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def plotly_plot(x,y, **kwargs):
        # if not np.isscalar(y):
        #     ys = y
        #     y = y[0]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='lines',hovertemplate=None))
        fig.update_layout(hovermode="x unified")
        fig.update_xaxes(title_text=None) #, tickformat = ".3s"
        fig.update_yaxes(title='y') #, tickformat = ".3s"
        for name in kwargs:
            if name == 'show_axis':
                if not kwargs[name]:
                    fig.update_xaxes(visible=False)
                    fig.update_yaxes(visible=False)
            if name == 'yrange':
                fig.update_yaxes(range=kwargs[name])
            if name == 'log_x':
                if kwargs[name]:
                    fig.update_xaxes(type="log")
            if name == 'log_y':
                if kwargs[name]:
                    fig.update_yaxes(type="log")
            if name == 'xticksuffix':
                fig.update_xaxes(ticksuffix=kwargs[name])
            if name == 'yticksuffix':
                fig.update_yaxes(ticksuffix=kwargs[name])
            if name == 'xtickformat':
                fig.update_xaxes(tickformat=kwargs[name])
            if name == 'ytickformat':
                fig.update_yaxes(tickformat=kwargs[name])
            if name == 'xtitle_text':
                fig.update_xaxes(title_text=kwargs[name])   
            if name == 'ytitle_text':
                fig.update_yaxes(title_text=kwargs[name])

        # if not np.isscalar(ys):
        #     for i in range(1,len(ys)):
        #         fig.add_trace(go.Scatter(x=x, y=ys[i], mode='lines', name='lines'))
        fig.show()

    def plotly_plot2(x,ys, **kwargs): #Started to plot multiple charts
        fig = make_subplots(rows=len(ys), cols=1)

        for i,y in enumerate(ys):
            fig.add_trace(go.Scatter(x=x, y=y, mode='lines',hovertemplate=None),row=i+1,col=1)
            fig.update_xaxes(type='log', row=i+1, col=1, title='Frequency in rad/s',tickformat = ".0e")

        fig.update_yaxes(title='Magnitude in dB', row=1, col=1)
        fig.update_yaxes(title='Phase in °', row=2, col=1)


        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
   #         height=1000/8*suplotnumber_max,
            width=800,
        )

 #       fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def plotly_plot3(x,ys, **kwargs): #Used for Bode Plot
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(go.Scatter(x=x, y=ys[0],name='Magnitude in dB', mode='lines'),secondary_y=False,)
        fig.add_trace(go.Scatter(x=x, y=ys[1],name='Phase in °', mode='lines'),secondary_y=True,)
        fig.update_xaxes(type='log', row=1, col=1, title='Frequency in rad/s',tickformat = ".0e")

        fig.update_yaxes(title_text='Magnitude in dB', secondary_y=False)
        fig.update_yaxes(title_text='Phase in °', secondary_y=True)


        fig.update_layout(
            hovermode="x unified",
            legend_title=None,

            font=dict(
                family="Arial",
                size=12,
                color="black"
            ),
            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.15,
                xanchor="left",
                x=0.01
            ),
   #         height=1000/8*suplotnumber_max,
            width=800,
        )

 #       fig.write_image(filepath.replace('.csv','.svg'))
        fig.show()

    def symTF_to_sciTF(symTF):
        #convert sympy transfer function to scipy system
        if str(symTF.num).find('s') == -1:
            num = [symTF.num]
        else:
            num = Poly(symTF.num).all_coeffs()

        if str(symTF.den).find('s') == -1:
            den = [symTF.den]
        else:
            den = Poly(symTF.den).all_coeffs()

        #convert sympy numbers to float
        num = [float(i) for i in num]
        den = [float(i) for i in den]

        sciTF = signal.TransferFunction(num, den)
        return sciTF


In diesem Teil des Skriptums geht es darum wie wir Maschinen und Schaltungen dazu bringen, trotz Störeinflüssen das gewünsche Verhalten zu Zeigen. Zum Beipiel soll ein Tempomant des Autos die Geschwindigkeit halten, trotz starkem Gegenwindes. Es werden die Grundlagen der Regelungstechnik vermittelt. Dabei wird das theoretische Wissen anhand konkreter Anwendungen erarbeitet.

## Warum wir regeln
<div style="text-align: justify">
    
Viele Aufgaben von Maschinen können auch durch Steuern umgesetzt werden. Eine Regelung erlaubt es aber, auf unerwünschte Einflüsse, sogenannte Störgrößen, zu reagieren. Als Beispiel soll der Tempomat, Geschwindigkeitsregelanlage, des Autos dienen. Die Aufgabe des Tempomates ist es, die Geschwindigkeit, Regelgröße, konstant zu halten. Als unerwünschte Einflüsse, Störgrößen, sind alle physikalischen Größen zu betrachten, welche die Geschwindigkeit beeinflussen. Beispiele sind die Steigung der Straße und Wind.  
Die Geschwindigkeit des Autos wird über die Leistung, Stellgröße, bestimmt. Führt die Straße Bergauf wird mehr Leistung für die gleiche Geschwindigkeit benötigt. Es muss also die Leistung laufend angepasst werden, um eine konstante Geschwindigkeit zu erhalten.  
Bei einer Steuerung würde eine Leistung eingestellt werden und sich daraus eine Geschwindigkeit ergeben. Dieses wäre jedoch nur für einen voreingestellten Fall identisch mit der gewünschten Geschwindigkeit.   

</div>

## Wie wir regeln - Der Standardregelkreis
<div style="text-align: justify">
Regeln ist ein Vorgang, bei dem der IST-Wert einer Größe gemessen und, durch Nachstellen der Stellgröße, dem SOLL-Wert angeglichen wird.  
Dazu wird das Ergebnis an den Eingang zurück geführt und vom Sollwert subtrahiert. Es entsteht eine Rückkopplung. Durch das negative Vorzeichen handelt es sich um eine Rückkopplung im Spezialfall einer Gegenkopplung. Die Differenz aus dem Sollwert und dem zurückgeführten Istwert ist die sogenannte Regelabweichung welche über den Regler zur Stellgröße wird. Die Stellgröße ist nun die physikalische Größe die die Regelstrecke zum gewünschten Verhalten führt.
    
</div>

![Standardregelkreis](images/Vollsändiger_Regelkreis){#fig-Regelkreis}

### Reglertypen
Es kann zwischen zwei Arten von Reglern unterschieden werden. Erstere sind einfache Regler die die Stellgröße nur zwischen verschiedenen Zuständen hin und her Schalten können. Zum Beispiel Ein / Aus. Oder die Gänge eines Automatikgetriebes. Diese Regler werden **unstetige Regler** genannt. Unstetige Regler können gut mittels Hysteresen beschrieben werden.  
Der zweite Typ von Regler kann die Stellgröße kontinurierlich anpassen. Diese Regler werden **stetige Regler** genannt. Stetige Regler können gut mit mathematische Gleichungen im Laplacebereich beschrieben werden.  

## Unstetige Regler
Klassische unstetige Regler sind Bimetallschalter. Diese werden zum Beipiel bei Heizlüftern eingesetzt.  
Ist der Raum, und damit das Bimetall kalt so ist der Kontakt geschlossen und der Lüfter läuft. Wird die Raumluft und damit das Bimetall warm wird der Kontakt geöffnet und der Lüfter hört auf zu heizen.

In [ ]:
#| code-fold: true
#| echo: false

T = MySymbol('T',description='Temperatur')
Tref = MySymbol('T_ref',description='Referenz Temperatur')
deltaTein = MySymbol('T_ein',description='Einschaltschwelle')
deltaTaus = MySymbol('T_aus',description='Ausschaltschwelle')

T.value = np.arange(15,25,0.01)
Tref.value = 20
deltaTaus.value = 2
deltaTein.value = -2

QBookHelpers.print_description({T,Tref,deltaTein,deltaTaus})

In [ ]:
#| code-fold: true
#| label: fig-zweipunktreglerplot
#| fig-cap: Zweipunktregler eines einfachen Heizlüfters
#| echo: false

plt.rcParams['text.usetex'] = true
fig, ax = plt.subplots()
ax.plot(T.value,-np.sign(T.value-Tref.value-deltaTaus.value),'b',alpha=0.8)
ax.plot(T.value,-np.sign(T.value-Tref.value-deltaTein.value),'r',alpha=0.8)
ax.set_xlabel('Temperatur in °C')
ax.set_ylabel('Schaltzustand')
ax.set_yticks([-1,1])
ax.set_yticklabels(['Off','On'])
ax.set_xticks([Tref.value,Tref.value+deltaTaus.value,Tref.value+deltaTein.value])
ax.set_xticklabels(['$' + latex(Tref) + '$','$T_{off}$','$T_{on}$'])

plt.grid()


### Zweipunktregler
Der Zweipunktregler kann, wie der Name schon sagt, die Stellgröße zwischen zwei Zuständen schalten. Zum Beipiel die Heizung einschalten wenn die Temperatur zu niedrig ist und wieder Abschalten wenn die Temperatur hoch genug ist.  Siehe dazu die Kennlinie @fig-zweipunktreglerplot.
Die Kennlinie stellt eine Hysterese dar. Die Umsetzung ist auch mittels Operationsverstärker möglich.


## Stetige Regler 
Für das Verständnis von stetigen Reglern ist es hilfreich die Regelungstechnik mathematisch zu betrachten, da sich ein Regler sehr gut mit Formeln beschreiben und erklären lässt. In einem eigenen Kapitel soll behandelt werden wie Regler praxisnahe implementiert werden können.  
Der oben gezeigte Regelkreis, @fig-Regelkreis, lässt sich mathematisch als Übertragungsfunktion beschreiben. Hier werden ausschließlich SISO (Single Input Single Output) und LTI (Linear Time Invariant) Systeme betrachtet. Das Bedeutet Systeme die einen Eingang und einen Ausgang haben. Jeder Block kann einzeln mit einer Übertragungsfunktion, analog der Vierpoltheorie, beschrieben werden. Wie auch in der Vierpoltheorie kann aber auch eine Verschaltung von Blöcken als Übertragungsfunktion beschrieben werden. Ein Block wird in der Regelungstechnik auch als **Strecke** bezeichnet.

### Die Übertragungsfunktion
#### Motivation
Die Übertragungsfunktion Beschreibt den Zusammenhang zwischen Ausgang und Eingang.  
Ist die Übertragungsfunktion bekannt, so kann die Strecke und deren Verhalten (der Ausgang) auf verschiedene Eingänge berechnet werden. Dies wird auch Simulation genannt. Für Marketingzwecke könnte die Übertragungsfunktion auch als einfache Form eines "digitalen Zwillings" bezeichnet werden.
Ist die Übertragungsfunktion mathematisch beschrieben, können Regler entworfen und getestet werden, ohne das tatsächliche physikalische Modell benutzen zu müssen. Dies ist speziell sinnvoll wenn, das physikalische System für Testzwecke nicht zur Verfügung steht bzw. nicht für Testzwecke geeignet ist.
Eine Strecke (=physikalisches System) steht z.B. nicht zur Verfügung, wenn: 

* Es sich geographisch woanders befindet  
* Es für die Produktion benötigt wird  
* Es noch nicht gebaut wurde  

Eine Strecke (=physikalisches System) ist nicht geeignet für Testzwecke wenn z.B.:  

* Das System sehr langsam ist (Heizung eines Gebäudes)  
* Ein fehlerhafter Regler großen Schaden anrichten kann  

#### Streckenidentifikation {#sec-Streckenidentifikation}
Als Streckenidentifiaktion wird der Vorgang beschrieben, von einem physikalischem System das mathematische Modell, die Übertragungsfunktion, zu erstellen. Zwei Methoden wie dieses Ziel erreicht werden kann, werden hier beschrieben.  

* Der mathematische Ansatz, @sec-MathStreckenidentifikation  
* Der messtechnische Ansatz, @sec-MessStreckenidentifikation  


In [ ]:
#| code-fold: true
E = MySymbol('E',description = 'Eingang')
V = MySymbol('V',description = 'Verarbeitung, die Übertragungsfunktion')
A = MySymbol('A',description = 'Ausgang')
G = MySymbol('G',description = 'Allgemeine Übertragungsfunktion')

Veq = Eq(V,A/E)

display(Markdown(f'$$\n{latex(Veq)}\n$${{#eq-EVA}}'))

QBookHelpers.print_description(Veq.free_symbols)

Gängige Bezeichnungen der Übertragungsfunktion der einzelnen Blöcke ist wie folgt.

In [ ]:
#| code-fold: true
G = MySymbol('G',description = 'Übertragungsfunktion der zu Regelnden Strecke')
R = MySymbol('R',description = 'Übertragungsfunktion des Reglers')
M = MySymbol('M',description = 'Übertragungsfunktion des Sensors')

QBookHelpers.print_description({G,R,M})

### Mathematische Streckenidentifikation {#sec-MathStreckenidentifikation}
Aus den physikalischen Zusammenhängen kann die Übertragungsfunktion berechnet werden.
Die komplexe Schreibweise ist nur für periodische sinusförmige Signale geeignet. Sollen Signale betrachtet werden die beliebig, stetig, sind ist die komplexe schreibweise nicht ausreichend. Es müssen die physikalischen Gleichungen in differentieller Form angeschrieben werden.

Um die Mathematik möglichst einfach zu halten wird in der Regelungstechnik im Laplace Bereich gearbeitet. Dadurch ist  es nicht notwendig die Diffenrentialgleichung bei physikalischen Systemen, die durch eine Differentialgleichung beschrieben werden, zu lösen.

Ein Besipiel, wie eine mathematische Streckenidentifikation abläuft ist in Abschnitt @sec-Laplace zu finden.

### Messtechnische Streckenidentifikation {#sec-MessStreckenidentifikation}
Manche Systeme sind zu komplex um Sie mathematisch zu beschreiben. Andere Systeme sind mathematisch nicht beschreibbar, weil das Innenleben nicht bekannt ist. In diesen Fällen kann die messtechnische Ermittlung der Übertragungsfunktion herangezogen werden.  
Dabei wird am Eingang der Strecke ein Testssignal aufgeschalten und der Ausgang gemessen. Aus diesen Messdaten kann die Übertragungsfunktion, ein mathemtisches Modell, der Strecke erstellt werden.

### Die Laplace Transformation {#sec-Laplace}
oder die Anstrengung der Faulen.

#### Warum Laplace
Um eine Übertragungsfunktion zu Berechnen muss der Ausgang durch den Eingang dividiert werden. Wird das physikalische System durch eine lineare Gleichung beschrieben ist das sehr Einfach möglich und die Laplace Transformation ist nicht notwendig. Ein Beispiel dafür is das Ohm'sche Gesetz.

In [ ]:
#| code-fold: true
U_ohm = MySymbol('U',description = 'Spannung am Widerstand als Ausgang')
R_ohm = MySymbol('R_ohm',description = "Ohm'scher Widerstand als Übertragungsfunktion")
I_ohm = MySymbol('I',description = 'Strom am Widerstand als Eingang')

R_ohm_eq=Eq(R_ohm,U_ohm/I_ohm)

display(Markdown(f'$$\n{latex(R_ohm_eq)}\n$${{#eq-Ohm_eq}}'))

QBookHelpers.print_description(R_ohm_eq.free_symbols)

Wird das physikalische System aber durch eine Differentialgleichung beschrieben, wie zum Beispiel bei einem Tiefpass, so wäre es notwendig zuerst die Differentialgleichung zu lösen um die Übertragungsfunktion zu berechnen. Hier bietet die Lapalce Transformation eine erhebliche erleichterung.  
Wird die Übertragungsfunktion im Laplace-Bereich angeschrieben, ergeben sich weitere Vorteile, wenn es später darum geht einen Regler zu entwerfen und die Stabilität einer Strecke zu beurteilen.  

##### Beispiel Übertragungsfunktion eines Tiefapsses

![Tiefpass](images/Tiefpass){#fig-Tiefpass}


In [ ]:
#| code-fold: true
C_cap = MySymbol('C',description = 'Kapazität')
#ut = MyFunction('u(t)',description = 'Spannung')

t = MySymbol('t',description = 'Zeit')
uct0 = MySymbol('u_{C}(0)',description = 'Spannung am Kondensator zum Zeitpunkz *t* = 0 s')
R_ohm.description = 'Ohmscher Widerstand'
ict = Function('i_c',description = 'Strom')(t)
uint = Function('u_in',description = 'Eingangsspannung')(t)
uct = Function('u_c',description = 'Ausgangsspannung')(t)

ict_eq1=Eq(ict,1/C_cap*diff(uct,t))

#uct_eq = Eq(integrate(ict_eq.rhs,t)*C_cap,integrate(ict_eq.lhs,t)*C_cap + uct0)

ict_eq2 = Eq(ict,1/R_ohm*(uint-uct))
QBookHelpers.print_equation(ict_eq1,label='eq-ict_eq1')
QBookHelpers.print_equation(ict_eq2,label='eq-ict_eq2')

Durch Gleichsetzten von  @eq-ict_eq1 und @eq-ict_eq2 ergibt sich die allgemeine Differenzialgleichung 1. Ordnung für den Tiefpass.


In [ ]:
#| code-fold: true


RCTP_eq = Eq(diff(uct,t)+1/(R_ohm*C_cap)*uct,1/(R_ohm*C_cap)*uint)

QBookHelpers.print_equation(RCTP_eq,label='eq-RCTP')
QBookHelpers.print_description(RCTP_eq.free_symbols)
QBookHelpers.print_description({ict,uct,uint})

Müsste nun von dieser Differentialgleichung die Übertragungsfunktion, also $G=Ausgang/Eingang$, angegeben werden, so müsste zunächst die Differentialgleichung gelöst werden.  
Die Laplace Transformation bietet hier einen alternativen Weg der mit weiteren Vorteilen verbunden ist wenn es darum geht Blöcke miteinander zu kombinieren oder Aussagen über das System zu treffen.

#### Laplacetransformation
Die tiefere Mathematik der Laplacetransofrmation überlassen wir hier den Mathematiker:innen und den ersten Semstern eines Studiums. Wir wollen die Laplacetransformation lediglich als Werkzeug zur vereinfachung unserer Arbeit verwenden. Dazu benötigen wir folgende Grundregeln.  
Vereinfacht ist die Laplacetransformation als eine Übersetzung aus dem Zeitbereich, also mit der varaible $t$, in den Frequenzbereich mit der Variable $s$ zu verstehen. Die Übersetzung erfolgt in vielen Fällen sehr einfach mittels Tabelle. Hier wird die Transformation nur für ausgewählte Signale und mathematische Operationen angeführt.  


| Zeitbereich x(t) | Frequenzbereich X(s) | Bemerkung |
| :---             | :------ |:----------|
| $\frac{d \ x(t)}{d \ t}$ |$s \cdot X(s) - x(0)$ |Transformation der Ableitung nach der Zeit, $x(0)$ ist dabei der Wert zum Zeitpunkt Null. Bei einem Kondensator wäre dies zum Beispiel der Ladezustand zu Beginn.|
| ${ \int x(t) \, d \ t}$  |$\frac{1}{s} \cdot X(s)$  |Transformation der Integration über der Zeit|
| $\delta (t)$ | $1$ | Transformation des Impulses|
| $\sigma (t)$ | $\frac{1}{s}$| Transformation des Sprunges|
| $e^{at}$| $\frac{1}{s -a}$| |
| $\frac{1}{a} e^{\frac{-t}{a}}$| $\frac{1}{1 + as}$| |

:Laplacetransformationstabelle {#tbl-laplace}

Wird nun Gleichung @eq-RCTP mittels der Tabelle @tbl-laplace transformiert erhalten wir eine Gleichung aus der wir durch einfaches Umformen eine Übertragungsfunktion erhalten. Es wird angenommen, dass $ x(0) = 0 $ ist.

In [ ]:
#| code-fold: true

from sympy.abc import s, p, a

from sympy.physics.control.lti import TransferFunction

UC = MySymbol('U_{C}',description = 'Ausgangsspannng im Laplacebereich')
UIN = MySymbol('U_{IN}',description = 'Eingangsspannung im Laplacebereich')

RCTPLap_eq = Eq(s*UC+1/(R_ohm*C_cap)*UC,1/(R_ohm*C_cap)*UIN)
QBookHelpers.print_equation(RCTPLap_eq)

RCTPLap_eq = Eq(G,UC/UIN)
QBookHelpers.print_equation(RCTPLap_eq)

tf1 = Eq(G,TransferFunction(1, s + R_ohm*C_cap, s))
QBookHelpers.print_equation(tf1)


### Testsignale und Streckenverhalten {#sec-TestStrecke}

#### Testsignale

#### Streckenverhalten
##### Interaktiver PT2 Simulator

```{ojs}
//| echo: false

Plot.plot({
  y: {
    grid: true,
  //  domain: [0, 4]
  },
  marks: [
    Plot.line(y, {
      x: "t",
      y: "s",
      stroke: '#888'
    }),
    Plot.line(y, { x: "t", y: "u", stroke: '#34f' })
  ]
})

viewof input = Select(
  ['step', 'hardstep', 'smoothstep', 'ramp', 'quadratic ramp'],
  {
    value: 'step',
    label: 'Input'
  }
)

//viewof Kp = Range([1e-12, 100], {
//  label: tex`K_P`,
//  value: 1,
//  transform: Math.log
//})

//viewof Ki = Range([1e-12, 10], {
//  label: tex`K_I`,
//  value: 0,
//  transform: Math.log
//})

//viewof Kd = Range([1e-12, 1], {
//  label: tex`K_d`,
//  value: 0,
//  transform: Math.log
//})

//viewof dt = Range([1e-4, 0.1], {
//  value: 0.001,
//  transform: Math.log,
//  label: tex`\Delta t`
//})

viewof Kpt2 = Range([1e-12, 5], {
  value: 1,
  transform: Math.log,
  label: tex`K_{pt2}`
})

viewof D = Range([1e-12, 10], {
  value: 0.5,
  transform: Math.log,
  label: tex`D`
})

viewof T = Range([1e-12, 5], {
  value: 1,
  transform: Math.log,
  label: tex`T`
})

viewof tMax = Range([1, 10e3], {
  value: 10,
  transform: Math.log,
  label: tex`t_{max}`
})

y = {
  const output = [];

  let p; 
  let t0 = -0.5;
  //let tMax = 20;

  let t = t0;
  let d = 0;
  let u = 0;
  let i = 0;
  let e = 0;
  let eprev = e;
  let pu = u;
  let ppu = u;
  let du = 0;
  let ddu = 0;
  let pdu = 0;
  let y = 0;
  let py = 0;
  let ppy = 0;
  let dy = 0;
  let ddy = 0;
  let pdy = 0;
  let de = 0;
  let dt = 0.001;
  let s = setPoint(t0);
  output.push({ t, u, s });

  let j = 1;
  while (t < tMax) {
    t = t0 + j * dt;

    s = setPoint(t);
   
    e = s - u;
    de = eprev-e;

    //du = (u-pu)/dt;
    //ddu = (du - pdu)/dt;

    //dy = (py - y)/dt;
    //ddy = (dy - pdy)/dt;

    p = e;
    i += e * dt;
    d = de/dt;

    y = s; //(Kp * p + Ki * i + Kd * d) * dt;
    
    u = (2*D*T*dt*pu + Kpt2*dt**2*y - T**2*ppu + 2*T**2*pu)/(2*D*T*dt + T**2 + dt**2);

    ppu = pu;
    pu = u;
    //pdu = du;

    //ppy = py;
    //py = y;
    //pdy = dy;

    eprev = e;

    output.push({ t, u, s });
    j++;
  }

  return output;
}

setPoint = {
  function hardstep(x) {
    return Math.max(0, Math.min(1, x));
  }
  function smoothstep(x) {
    var x = hardstep(x);
    return x * x * (3 - 2 * x);
  }
  switch (input) {
    case 'step':
      return t => (t >= 0 ? 1 : 0);
    case 'ramp':
      return t => (t >= 0 ? t : 0);
    case 'quadratic ramp':
      return t => (t >= 0 ? t * t : 0);
    case 'smoothstep':
      return smoothstep;
    case 'hardstep':
      return hardstep;
    default:
      throw new Error('Invalid input');
  }
}

import { Range, Select } from '@observablehq/inputs'
```



#### Identifikation




### Zusammenschaltung von Blöcken
Werden Blöcke kombiniert können die resultierenden Übertragungsfunktionen berechnet werden.  
Zur vereinfachung kann die Übertragungsfunktion des Sensors mit $M=1$ angenommen werden, $M=1$, wenn dieser im Verhälnis zur Strecke und zum Regler vernachlässigbar ist. Dies ist zum Beispiel der Fall wenn der Sensor viel schneller ist als die Strecke und der Regler. Diese Vorraussetzung ist für viele Systeme gegeben.  

Für den Regelkreis, @fig-Regelkreis, ergebn sich folgende Möglichkeiten.

#### Die Führungsübertragungsfunktion
Die Führungsübertragungsfunktion gibt das Verhältnis zwischen Sollgröße und Istgröße an. Sie Beschreibt damit das Verhalten des Regelkreises mit der Sollgröße als Eingang und der Istgröße als Ausgang. Ist eine Regelstrecke ideal so ist die die Führungsübertragungsfunktion gleich Eins.

In [ ]:
#| code-fold: true
Fw = MySymbol('F_w',description = 'Führungsübertragungsfunktion')

Fw_eq=Eq(Fw,R*G/(1+R*G*M))

display(Markdown(f'$$\n{latex(Fw_eq)}\n$${{#eq-Fw_eq}}'))

QBookHelpers.print_description({Fw})

#### Die Schleifenübertragungsfunktion
Die Schleifenübertragungsfunktion ist die Übertragungsfunktion des offenen Regelkreises, also ohne Rückkopplung und ist im Laplace Bereich eine einfache Multiplikation.

In [ ]:
#| code-fold: true
Fo = MySymbol('F_o',description = 'Schleifenübertragungsfunktion')

Fo_eq=Eq(Fo,R*G)

QBookHelpers.print_equation(Fo_eq)
QBookHelpers.print_description({Fo})

#### Die Störübertragungsfunktion
Die Störübertragungsfunktion beschreibt wie sich die Störgröße auf den Ausgang auswirkt.

In [ ]:
#| code-fold: true
Fs = MySymbol('F_s',description = 'Störübertragungsfunktion')

Fs_eq=Eq(Fs,G/(1+Fo))

QBookHelpers.print_equation(Fs_eq)
QBookHelpers.print_description({Fs})

### Interaktiver PID Simulator

<span style="color:#888"> Eingangssignal $w$ </span>  
<span style="color:#34f"> Ausgangssignal $y$ </span>  
<span style="color:#fb2f03"> Regelabweichung $e$ </span>  
<span style="color:#d7fb03"> Stellgröße $u$ </span>  


```{ojs}
// The basic code layout was found on source: https://observablehq.com/@mbostock/inputs
// be aware, the code variables do not yet match the variable names in the document

Plot.plot({
  y: {
    grid: true,
  //  domain: [0, 4]
  },
  marks: [
    Plot.line(y_PID, {
      x: "t_PID",
      y: "s_PID",
      stroke: '#888'
    }),
    Plot.line(y_PID, { x: "t_PID", y: "u_PID", stroke: '#34f' }), //BLue
    Plot.line(y_PID, { x: "t_PID", y: "e_PID", stroke: '#fb2f03' }), //RED
    Plot.line(y_PID, { x: "t_PID", y: "y_PID", stroke: '#d7fb03' }),  //Greenisch
  ]
})
```


Reglereinstellungen:
```{ojs}

viewof Kp_PID = Range([1e-12, 10e3], {
  label: tex`K_P`,
  value: 1,
  transform: Math.log
})

viewof Ki_PID = Range([1e-12, 10e2], {
  label: tex`K_I`,
  value: 0,
  transform: Math.log
})

viewof Kd_PID = Range([1e-12, 10e1], {
  label: tex`K_d`,
  value: 0,
  transform: Math.log
})

//viewof dt = Range([1e-4, 0.1], {
//  value: 0.001,
//  transform: Math.log,
//  label: tex`\Delta t`
//})

```


Streckeneinstellungen:
```{ojs}

viewof Kpt2_PID = Range([1e-12, 10e3], {
  value: 1,
  transform: Math.log,
  label: tex`K_{pt2}`
})

viewof D_PID = Range([1e-12, 10], {
  value: 1,
  transform: Math.log,
  label: tex`D`
})

viewof T_PID = Range([1e-12, 5], {
  value: 1,
  transform: Math.log,
  label: tex`T`
})

```

Auswahl Eingangssignal:

```{ojs}

viewof input_PID = Select(
  ['step', 'hardstep', 'smoothstep', 'ramp', 'quadratic ramp'],
  {
    value: 'step',
    label: 'Input'
  }
)

```

Simulationseinstellungen:
```{ojs}

viewof tMax_PID = Range([1, 10e3], {
  value: 10,
  transform: Math.log,
  label: tex`t_{max}`
})

y_PID = {
  const output_PID = [];

  let p_PID; 
  let t0_PID = -0.5;
  //let tMax_PID = 20;

  let t_PID = t0_PID;
  let d_PID = 0;
  let u_PID = 0;
  let i_PID = 0;
  let e_PID = 0;
  let eprev_PID = e_PID;
  let pu_PID = u_PID;
  let ppu_PID = u_PID;
  let du_PID = 0;
  let ddu_PID = 0;
  let pdu_PID = 0;
  let y_PID = 0;
  let py_PID = 0;
  let ppy_PID = 0;
  let dy_PID = 0;
  let ddy_PID = 0;
  let pdy_PID = 0;
  let de_PID = 0;
  let dt_PID = 0.001;
  let s_PID = setPoint(t0_PID);
  output_PID.push({ t_PID, u_PID, s_PID });

  let j_PID = 1;
  while (t_PID < tMax_PID) {
    t_PID = t0_PID + j_PID * dt_PID;

    s_PID = setPoint_PID(t_PID);
   
    e_PID = s_PID - u_PID;
    de_PID = eprev_PID-e_PID;

    //du = (u-pu)/dt;
    //ddu = (du - pdu)/dt;

    //dy = (py - y)/dt;
    //ddy = (dy - pdy)/dt;

    p_PID = e_PID;
    i_PID += e_PID * dt_PID;
    d_PID = de_PID/dt_PID;

    y_PID = (Kp_PID * p_PID + Ki_PID * i_PID + Kd_PID * d_PID);
    
    u_PID = (2*D_PID*T_PID*dt_PID*pu_PID + Kpt2_PID*dt_PID**2*y_PID - T_PID**2*ppu_PID + 2*T_PID**2*pu_PID)/(2*D_PID*T_PID*dt_PID + T_PID**2 + dt_PID**2);

    ppu_PID = pu_PID;
    pu_PID = u_PID;
    //pdu = du;

    //ppy = py;
    //py = y;
    //pdy = dy;

    eprev_PID = e_PID;

    output_PID.push({ t_PID, u_PID, s_PID, e_PID, y_PID });
    j_PID++;
  }

  return output_PID;
}

setPoint_PID = {
  function hardstep_PID(x) {
    return Math.max(0, Math.min(1, x));
  }
  function smoothstep_PID(x) {
    var x = hardstep_PID(x);
    return x * x * (3 - 2 * x);
  }
  switch (input) {
    case 'step':
      return t_PID => (t_PID >= 0 ? 1 : 0);
    case 'ramp':
      return t_PID => (t_PID >= 0 ? t_PID : 0);
    case 'quadratic ramp':
      return t_PID => (t_PID >= 0 ? t_PID * t_PID : 0);
    case 'smoothstep':
      return smoothstep_PID;
    case 'hardstep':
      return hardstep_PID;
    default:
      throw new Error('Invalid input');
  }
}

//import { Range, Select } from '@observablehq/inputs'
```
